In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
ven_info = pd.read_csv("image_data/venomous_status_metadata.csv", names=["nid", "class_id", "ven"], header=0)
train_info = pd.read_csv("image_data/train_images_metadata.csv", index_col=0)
relevant = train_info[["image_path", "class_id",]].merge(ven_info, on="class_id")
n_classes = len(ven_info)

In [19]:
BATCH_SIZE = 64
def load_and_preprocess1(img_path, y1, img_size=(240, 240), onehot=True):
    img = tf.io.read_file("image_data/train_images_small/" + img_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, img_size)
    #img = tf.cast(img, tf.float32) / 255.0
    if onehot: y1 = tf.one_hot(y1, depth=n_classes)
    return img, y1

def make_dataset1(df, what):
    slices = (df['image_path'].values, df[what].values)
    ds = tf.data.Dataset.from_tensor_slices(slices)
    lp = load_and_preprocess1
    if what=="ven": lp = lambda x, y: load_and_preprocess1(x, y, onehot=False)
    ds = ds.map(lp, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    return ds

In [20]:
train_paths, test_val_paths = train_test_split(relevant, test_size=0.3, random_state=42) # does shuffle
val_paths, test_paths = train_test_split(test_val_paths, test_size=0.33, random_state=42) # does shuffle
cid_train_ds = make_dataset1(train_paths, "nid")
cid_val_ds = make_dataset1(val_paths, "nid")
cid_test_ds = make_dataset1(test_paths, "nid")
ven_train_ds = make_dataset1(train_paths, "ven")
ven_val_ds = make_dataset1(val_paths, "ven")
ven_test_ds = make_dataset1(test_paths, "ven")

In [5]:
import matplotlib.pyplot as plt
def plot_training_graphs():
    plt.plot(history.history['class_id_accuracy'])
    plt.plot(history.history['val_class_id_accuracy'])
    plt.plot(history.history['ven_accuracy'])
    plt.plot(history.history['val_ven_accuracy'])
    plt.title('model acc')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['class_id_acc', 'val_class_id_acc', 'ven_acc', 'val_ven_acc'], loc='upper left')

In [6]:
from tensorflow.keras.layers import Flatten, Conv2D, Dense, Input, MaxPooling2D, Dropout, Concatenate
from tensorflow.keras import losses
from tensorflow.keras import Model

In [7]:
class WeightedBinaryCrossentropy(losses.BinaryCrossentropy):
    def call(self, y_true, y_pred):
        l = super().call(y_true, y_pred)
        weights = y_true * 0.8 + (1 - y_true) * 0.2
        return tf.reduce_mean(l * weights)

In [10]:
## separate 

inp1 = Input(shape=(240, 240, 3))
# Conv A
x1 = Conv2D(16, (5, 5), padding="valid")(inp1)
x1 = MaxPooling2D(pool_size=(5,5))(x1)
x1 = Conv2D(64, (3, 3), padding="valid")(x1)
x1 = MaxPooling2D(pool_size=(3,3))(x1)
x1 = Conv2D(32, (3, 3), padding="valid")(x1)
x1 = MaxPooling2D()(x1)
x1 = Flatten()(x1)

x1 = Dense(32, activation="relu")(x1)
x1 = Dense(32, activation="relu")(x1)
ven_output = Dense(1, activation="sigmoid", name='ven')(x1)


inp2 = Input(shape=(240, 240, 3))
# Conv B
x2 = Conv2D(16, (5, 5), padding="valid")(inp2)
x2 = MaxPooling2D(pool_size=(5,5))(x2)
x2 = Conv2D(64, (3, 3), padding="valid")(x2)
x2 = MaxPooling2D(pool_size=(3,3))(x2)
x2 = Conv2D(32, (3, 3), padding="valid")(x2)
x2 = MaxPooling2D()(x2)
x2 = Flatten()(x2)

x2 = Dense(32, activation="relu")(x2)
x2 = Dense(32, activation="relu")(x2)
class_output = Dense(n_classes, activation="sigmoid", name='class_id')(x2)

ven_model = Model(inputs=inp1, outputs=ven_output, name='ven_model')
cid_model = Model(inputs=inp2, outputs=class_output, name='cid_model')

ven_model.compile(
    optimizer= tf.keras.optimizers.Adam(),
    loss=WeightedBinaryCrossentropy(),
    metrics=['accuracy'])
cid_model.compile(
    optimizer= tf.keras.optimizers.Adam(),
    loss=losses.CategoricalCrossentropy(),
    metrics=['accuracy'])

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.applications import ConvNeXtTiny

# 1. Load the model without the classification head (include_top=False)
base_model = ConvNeXtTiny(
    weights='imagenet',
    include_top=False,  # This is crucial for feature extraction
    input_shape=(240, 240, 3) # Specify input size
)

# 2. Freeze the weights of the base model
# This prevents the pre-trained weights from being updated during training.
base_model.trainable = False

# 3. Build your new custom classification head
inputs = tf.keras.Input(shape=(240, 240, 3))
x = base_model(inputs, training=False) # Pass the inputs through the frozen base
x = layers.GlobalAveragePooling2D()(x) # Apply pooling
x1 = Dense(64, activation="relu")(x)
x1 = Dense(32, activation="relu")(x1)
class_output = Dense(n_classes, activation="softmax", name='class_id')(x1)

cid_model = Model(inputs=inputs, outputs=class_output, name='cid_model')

cid_model.compile(
    optimizer= tf.keras.optimizers.Adam(),
    loss=losses.CategoricalCrossentropy(),
    metrics=['accuracy'])

In [25]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.applications import ResNet50

# 1. Load the model without the classification head (include_top=False)
base_model = ResNet50(
    weights='imagenet',
    include_top=False,  # This is crucial for feature extraction
    input_shape=(240, 240, 3) # Specify input size
)

# 2. Freeze the weights of the base model
# This prevents the pre-trained weights from being updated during training.
base_model.trainable = False

# 3. Build your new custom classification head
inputs = tf.keras.Input(shape=(240, 240, 3))
x = base_model(inputs, training=False) # Pass the inputs through the frozen base
x = layers.GlobalAveragePooling2D()(x) # Apply pooling
x = Dense(128, activation="relu")(x)
class_output = Dense(n_classes, activation="softmax", name='class_id')(x)

cid_model = Model(inputs=inputs, outputs=class_output, name='cid_model')

cid_model.compile(
    optimizer= tf.keras.optimizers.Adam(),
    loss=losses.CategoricalCrossentropy(),
    metrics=['accuracy'])

In [ ]:
%%time
history = cid_model.fit(cid_train_ds, validation_data=cid_val_ds, epochs=5, verbose=True)

Epoch 1/5


In [ ]:
ven_model.evaluate(ven_test_ds)

26/26 ━━━━━━━━━━━━━━━━━━━━ 6s 239ms/step - accuracy: 0.5242 - loss: 0.2181


[0.21807317435741425, 0.5241641402244568]

In [ ]:
%%time
history = cid_model.fit(cid_train_ds, validation_data=cid_val_ds, epochs=3, verbose=True)

Epoch 1/3
143/182 ━━━━━━━━━━━━━━━━━━━━ 25s 642ms/step - accuracy: 0.4326 - loss: 11136.6466

2025-11-20 12:34:23.141665: W tensorflow/core/lib/png/png_io.cc:95] PNG warning: iCCP: extra compressed data


182/182 ━━━━━━━━━━━━━━━━━━━━ 132s 726ms/step - accuracy: 0.7550 - loss: 454028.0000 - val_accuracy: 0.8956 - val_loss: 3079024.2500
Epoch 2/3
143/182 ━━━━━━━━━━━━━━━━━━━━ 25s 652ms/step - accuracy: 0.8916 - loss: 17912041.7517

2025-11-20 12:36:36.856742: W tensorflow/core/lib/png/png_io.cc:95] PNG warning: iCCP: extra compressed data


182/182 ━━━━━━━━━━━━━━━━━━━━ 133s 730ms/step - accuracy: 0.8940 - loss: 86519640.0000 - val_accuracy: 0.8956 - val_loss: 313882272.0000
Epoch 3/3
143/182 ━━━━━━━━━━━━━━━━━━━━ 26s 675ms/step - accuracy: 0.8916 - loss: 562706241.6783

2025-11-20 12:38:53.134662: W tensorflow/core/lib/png/png_io.cc:95] PNG warning: iCCP: extra compressed data


182/182 ━━━━━━━━━━━━━━━━━━━━ 137s 754ms/step - accuracy: 0.8940 - loss: 1166290688.0000 - val_accuracy: 0.8956 - val_loss: 2586675456.0000
CPU times: user 46min 26s, sys: 13.5 s, total: 46min 40s
Wall time: 6min 42s


In [ ]:
cid_model.evaluate(cid_test_ds)

26/26 ━━━━━━━━━━━━━━━━━━━━ 6s 242ms/step - accuracy: 0.8980 - loss: 767027456.0000


[767027456.0, 0.898024320602417]